In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt

import random
import io
import re

%matplotlib inline
%config Completer.use_jedi = False

In [2]:
sentence_length = 40
windows_length  = 8

In [3]:
def process_text(filepath):
    text = open(filepath, 'r', encoding='utf8').read()
    text = text.lower()
    text = re.sub(r'[^а-я\s\d.!\?—-]+', '', text)
    text = re.sub(r'[!\?]+', '.', text)
    text = re.sub(r'\s+', ' ', text)
    chars = sorted(list(set(text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    text = [s.strip() for s in text.split('.') if len(s) > 0]
    text = [[char_to_int[c] for c in sentence] for sentence in text]
    text = keras.preprocessing.sequence.pad_sequences(text, maxlen=sentence_length - 1,
                                                      padding='post', truncating='post', value=char_to_int[' '])
    text = [np.append(s, char_to_int['.']) for s in text]
    return text, char_to_int

In [4]:
filepath = '../input/Москва-Петушки.txt'
text, text_dict = process_text(filepath)

In [5]:
dataX = []
dataY = []

for sentence in text:
    for i in range(0, sentence_length - windows_length, 1):
        seq_in = sentence[i:i + windows_length]
        seq_out = sentence[i + windows_length]
        dataX.append([c for c in seq_in])
        dataY.append(seq_out)

In [6]:
X = np_utils.to_categorical(dataX).reshape((len(dataX), windows_length * len(text_dict), 1))
y = np_utils.to_categorical(dataY)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=4, batch_size=128, callbacks=callbacks_list)

Epoch 1/4
413/788 [==============>...............] - ETA: 18:11 - loss: 2.7836

KeyboardInterrupt: 